# Práctico 1 - Parte 2

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

[Documentación de Pytorch](https://pytorch.org/docs/stable/index.html)

[Tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

En esta segunda parte se cargan datos reducidos en la parte 1. Esto con el fin de optimizar memoria.

## Importaciones

In [11]:
import bz2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from gensim import corpora
import pandas as pd
from tqdm.notebook import tqdm
from practico1_modulo import *

## Constantes

In [14]:
ARCHIVO_SET_DE_ENTRENAMIENTO = './data/training_set.csv'
ARCHIVO_SET_DE_ENTRENAMIENTO_REDUCIDO = './data/training_set_reduced.csv'
ARCHIVO_SET_DE_PRUEBA = './data/test_set.csv'
ARCHIVO_SET_DE_VALIDACION = './data/validation_set.csv'
ARCHIVO_DE_EMBEDDINGS = './data/SBW-vectors-300-min5.txt.bz2'
ARCHIVO_DICCIONARIO = './data/diccionario.txt'
EPOCHS = 5
VOCAB_SIZE = 1000 # Cantidad de palabras en el diccionario
MUESTRAS = 10000 # Cantidad de muestras que se toman del set de entrenamiento
                  # Si se configura valor 0 (cero) se toman todas las muestras
TOKENS_ESPECIALES = {'[relleno]': 0, '[desconocido]': 1}
VALOR_DE_RELLENO = 0
BATCH_SIZE = 100

## Carga de datos

In [3]:
df_entrenamiento = pd.read_csv(ARCHIVO_SET_DE_ENTRENAMIENTO_REDUCIDO)
if MUESTRAS > 0:
    titulos = df_entrenamiento.sample(MUESTRAS).title.to_list()
else:
    titulos = df_entrenamiento.title.to_list()
print('Datos cargados con éxito, a continuación una muestra de los datos.')
df_entrenamiento.head()

Datos cargados con éxito, a continuación una muestra de los datos.


,title,category
0,Casita Muñecas Barbies Pintadas,DOLLHOUSES
1,Neceser Cromado Holográfico,TOILETRY_BAGS
2,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS
3,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS
4,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES


## Procesamiento de los títulos

In [6]:
corpus_titulos = []
for titulo in tqdm(titulos, desc="Procesando títulos"):
    titulo_procesado = procesar_titulo(titulo)
    corpus_titulos.append(titulo_procesado)
# corpus_titulos contiene todos los títulos procesados.
# Cada título es una lista de palabras procesadas
print(f'Se han procesado {len(corpus_titulos)} títulos.\nMuestra de los 5 primeros:')
for i in range(5):
    print(corpus_titulos[i])

Procesando títulos:   0%|          | 0/10000 [00:00<?, ?it/s]

Se han procesado 10000 títulos.
Muestra de los 5 primeros:
['campera', 'salida', 'racing', 'club', 'slim', 'celeste']
['juego', 'disco', 'freno', 'fremax']
['cargador', 'netbook', 'exelente']
['rueda', 'giratoria', 'silla', 'escritorio', 'oficina']
['collar', 'mariposas']


## Construcción del diccionario a partir del corpus de títulos

In [7]:
# https://radimrehurek.com/gensim/corpora/dictionary.html
print('Generando diccionario...')
diccionario = corpora.Dictionary(corpus_titulos)
diccionario.filter_extremes(no_below=2, no_above=0.5, keep_n=VOCAB_SIZE)
diccionario.patch_with_special_tokens(TOKENS_ESPECIALES)
diccionario.compactify()
print(f'Se generó un diccionario de longitud {len(diccionario)} a partir de {diccionario.num_docs} documentos.')

# Guardado del diccionario en archivo de texto
diccionario.save_as_text(ARCHIVO_DICCIONARIO, sort_by_word=True)
print(f'Se guardó el diccionario en {ARCHIVO_DICCIONARIO}.')

Generando diccionario...
Se generó un diccionario de longitud 1002 a partir de 10000 documentos.
Se guardó el diccionario en ./data/diccionario.txt.


## Encoding de datos

In [8]:
encoded_titulos = []
for titulo in tqdm(corpus_titulos, desc='Encoding de títulos'):
    encoded_titulo = diccionario.doc2idx(titulo, unknown_word_index=1)
    encoded_titulos.append(encoded_titulo)
print('Se realizó el encoding de los títulos.\nMuestra de los 5 primeros:')
for i in range(5):
    print(encoded_titulos[i])

Encoding de títulos:   0%|          | 0/10000 [00:00<?, ?it/s]

Se realizó el encoding de los títulos.
Muestra de los 5 primeros:
[1000, 3, 2, 1, 4, 1001]
[7, 5, 6, 1]
[8, 1, 1]
[11, 1, 12, 9, 10]
[13, 1]


## Completamiento de datos

In [9]:
print('Completamiento de datos.')
longitudes_titulos = [len(titulo) for titulo in encoded_titulos]
longitud_maxima = max(longitudes_titulos)
print(f'El título más largo tiene {longitud_maxima} palabras/indices.')
print(f'Se rellenará con {VALOR_DE_RELLENO} los valores faltantes en los títulos que tengan menor longitud.')
data = [d[:ele] + [VALOR_DE_RELLENO] * (longitud_maxima - ele) for d, ele in zip(encoded_titulos, longitudes_titulos)]
for i in range(5):
    print(data[i])
X = torch.LongTensor(data)

Completamiento de datos.
El título más largo tiene 14 palabras/indices.
Se rellenará con 0 los valores faltantes en los títulos que tengan menor longitud.
[1000, 3, 2, 1, 4, 1001, 0, 0, 0, 0, 0, 0, 0, 0]
[7, 5, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[8, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[11, 1, 12, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[13, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Conversión de categorías a etiquetas

In [10]:
idx_to_target = sorted(df_entrenamiento["category"].unique())
target_to_idx = {t: i for i, t in enumerate(idx_to_target)}
def encode_target(target):
    # Convierte las categorías a etiquetas
    return target_to_idx[target]
categoria_etiquetada = [encode_target(t) for t in df_entrenamiento['category']]
print('Se etiquetaron las categorías.\nMuestra de las 5 primeras:')
for i in range(5):
    print(categoria_etiquetada[i])
y = torch.LongTensor(categoria_etiquetada)

torch.save(X, './data/X_train.pt')
torch.save(y, './data/y_train.pt')

Se etiquetaron las categorías.
Muestra de las 5 primeras:
188
570
116
25
103


## Embedding de títulos

In [15]:
# https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding
vector_size = 300
embeddings_matrix = torch.randn(len(diccionario), vector_size)
embeddings_matrix[0] = torch.zeros(vector_size)
print(f'Tamaño de la matriz de embeddings: {embeddings_matrix.shape}.')
with bz2.open(ARCHIVO_DE_EMBEDDINGS, mode='rt') as file:
    for line in tqdm(file, total=1000654, desc="Recorriendo archivo de embeddings"):
        word, vector = line.strip().split(None, 1)
        if word in diccionario.token2id:
            embeddings_matrix[diccionario.token2id[word]] = torch.FloatTensor([float(n) for n in vector.split()])
embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                          padding_idx=0)
print(f'Finalizado el embedding de tamaño {embeddings.weight.shape}.')

Tamaño de la matriz de embeddings: torch.Size([1002, 300]).


Recorriendo archivo de embeddings:   0%|          | 0/1000654 [00:00<?, ?it/s]

Finalizado el embedding de tamaño torch.Size([1002, 300]).


## Construcción del Dataset

El dataset se construye a partir del dataframe de Pandas que tiene dos columnas:
- **title**
- **category**

In [16]:
train_dataset = MeLiChallengeDataset(X, y)
print(len(train_dataset))
print(train_dataset[10])
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          drop_last=False)
i = 0
for data in tqdm(train_loader):
    i += 1
print(f'{i} iteraciones')

10000
{'data': tensor([21,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0]), 'target': tensor(1)}


  0%|          | 0/100 [00:00<?, ?it/s]

100 iteraciones


## Preprocesamiento de los datos

El preprocesamiento de texto tiene dos propósitos:
- Tokenizar los títulos (datos) de modo que se quiten los signos de puntuación y palabras cortas como preposiciones y conjunciones (stopwords), todas las palabras queden en minúsculas, se separen en listas de palabras, etc.
- Transformar las categorías en etiquetas numéricas.

## Carga del Dataset

## Construcción del Modelo

In [15]:
class MeLiChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with gzip.open(pretrained_embeddings_path, encode='utf-8', "rt") as fh:
#       with bz2.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] = torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 632)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

## Algoritmo de Optimización

In [16]:
%%time
model = MeLiChallengeClassifier(ARCHIVO_DE_EMBEDDINGS, train_processor.dictionary, 50, True)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

CPU times: user 1.69 s, sys: 53 ms, total: 1.74 s
Wall time: 1.76 s


## Entrenamiento del modelo

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Utilizando {device}')
model.to(device)

Utilizando cuda:0


MeLiChallengeClassifier(
  (embeddings): Embedding(5002, 50, padding_idx=0)
  (hidden1): Linear(in_features=50, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=632, bias=True)
)

In [18]:
# %%time
# for epoch in range(EPOCHS):  # Recorre el dataset multiples veces
#     model.train()
#     running_loss = 0.0
#     for data in train_loader:
#         inputs = data['data'].to(device)
#         labels = data['target'].to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = loss_function(outputs, labels.squeeze().long())
#         loss.backward()
#         optimizer.step()

In [19]:
# model.train()
# for data in train_loader:
#     inputs = data['data'].to(device)
#     target = data['target'].to(device)
#     optimizer.zero_grad()
#     output = model(inputs)
#     loss = loss_function(output, target)
#     loss.backward()
#     optimizer.step()

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader):
        X, y = data['data'].to(device), data['target'].to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## Evaluación del Modelo

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data in dataloader:
            X, y = data['data'].to(device), data['target'].to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_function, optimizer)
    test(test_loader, model, loss_function)
print("Done!")

Epoch 1
-------------------------------
loss: 6.448280  [    0/20000]
Test Error: 
 Accuracy: 0.1%, Avg loss: 6.448738 

Epoch 2
-------------------------------
loss: 6.448425  [    0/20000]
Test Error: 
 Accuracy: 0.1%, Avg loss: 6.448745 

Epoch 3
-------------------------------
loss: 6.448981  [    0/20000]
Test Error: 
 Accuracy: 0.1%, Avg loss: 6.448733 

Epoch 4
-------------------------------
loss: 6.448917  [    0/20000]
Test Error: 
 Accuracy: 0.1%, Avg loss: 6.448730 

Epoch 5
-------------------------------
loss: 6.448575  [    0/20000]
Test Error: 
 Accuracy: 0.1%, Avg loss: 6.448724 

Done!


In [23]:
# i = 0
# for batch, data in enumerate(train_loader):
#     i += 1
#     print (f'Lote {i}.')
#     if i%10 == 0:
#         print(type(data['data']))
#         print(type(data['target']))
# print(f'{i} iteraciones.')

## Guardado de los parámetros del modelo